In [13]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
import libraries.utils as utils
import libraries.magnetization as mag
import sklearn
import itertools

In [113]:
def psd(x):
    x = (-1) ** np.array(x)
    return np.abs(np.fft.rfft(x)[1:]) ** 2
def norm_psd(x):
    vec = psd(x)
    return vec / np.linalg.norm(vec)
# note all the data points have dc component = 0

In [3]:
J2s = np.linspace(0, 0.5-0.005, 100)

In [ ]:
threshold = 1e-12
N=12; J1=1; J2=0.3

gs = qt.qload(f'../J1J2_info/N{N}/n{N}gs_J2_{round(J2, 3)}')
states, signs = utils.get_nonzero_states(N, gs, threshold)
pred_signs =  [-1 + 2 * (mag.count_half_magnetization(i) % 2) for i in states]
deviations = utils.find_sign_deviations(states, signs, pred_signs)
print(len(deviations))

192


In [32]:
deviations = set(deviations)
nondev = set()
for s in states:
    if s not in deviations:
        nondev.add(s)
print(len(deviations), len(nondev), len(states))

192 732 924


In [33]:
psds = {s:psd(utils.generate_state_1D(s, N)) for s in states}
psds

{63: array([5.97128129e+01, 2.09541178e-31, 8.00000000e+00, 1.23259516e-32,
        4.28718708e+00, 0.00000000e+00]),
 95: array([44.78460969,  4.        ,  0.        , 12.        ,  3.21539031,
        16.        ]),
 111: array([33.85640646, 12.        ,  8.        , 12.        ,  6.14359354,
         0.        ]),
 119: array([2.98564065e+01, 1.60000000e+01, 1.60000000e+01, 4.93038066e-32,
        2.14359354e+00, 1.60000000e+01]),
 123: array([33.85640646, 12.        ,  8.        , 12.        ,  6.14359354,
         0.        ]),
 125: array([44.78460969,  4.        ,  0.        , 12.        ,  3.21539031,
        16.        ]),
 126: array([5.97128129e+01, 4.93038066e-32, 8.00000000e+00, 4.93038066e-32,
        4.28718708e+00, 0.00000000e+00]),
 159: array([33.85640646, 12.        ,  8.        , 12.        ,  6.14359354,
         0.        ]),
 175: array([24., 16.,  0.,  0., 24., 16.]),
 183: array([20., 12.,  8., 12., 20.,  0.]),
 187: array([22.92820323,  4.        , 16.        

In [127]:
norm_psds = {s: psds[s] / np.linalg.norm(psds[s]) for s in psds}

In [34]:
np.linalg.norm(np.array([1, 0, 1, 2] - np.array([4, 2, 1, 1])))

np.float64(3.7416573867739413)

In [35]:
pairs = list(itertools.combinations(states, 2))
print(len(pairs))
# n is negative (no deviatoin), p is positive (deviation)
n_n = []
n_p = []
p_p = []
for s1, s2 in pairs:
    dist = np.linalg.norm(psds[s1] - psds[s2])
    if s1 in deviations and s2 in deviations:
        p_p.append(dist)
    elif s1 not in deviations and s2 not in deviations:
        n_n.append(dist)
    else:
        n_p.append(dist)

426426


In [36]:
print(f'avg nondev-nondev dist: {np.mean(n_n)}')
print(f'avg dev-dev dist: {np.mean(p_p)}')
print(f'avg nondev-dev dist {np.mean(n_p)}')
print(f'std nondev-nondev dist: {np.std(n_n)}')
print(f'std dev-dev dist: {np.std(p_p)}')
print(f'std nondev-dev dist {np.std(n_p)}')

avg nondev-nondev dist: 42.90877673505551
avg dev-dev dist: 23.16495516086934
avg nondev-dev dist 42.05447296467072
std nondev-nondev dist: 19.17854136709876
std dev-dev dist: 12.529861046314089
std nondev-dev dist 14.191142194289126


In [95]:
from sklearn.cluster import KMeans

In [96]:
np.array(list(psds.values()))

array([[5.97128129e+01, 2.09541178e-31, 8.00000000e+00, 1.23259516e-32,
        4.28718708e+00, 0.00000000e+00],
       [4.47846097e+01, 4.00000000e+00, 0.00000000e+00, 1.20000000e+01,
        3.21539031e+00, 1.60000000e+01],
       [3.38564065e+01, 1.20000000e+01, 8.00000000e+00, 1.20000000e+01,
        6.14359354e+00, 0.00000000e+00],
       ...,
       [3.38564065e+01, 1.20000000e+01, 8.00000000e+00, 1.20000000e+01,
        6.14359354e+00, 0.00000000e+00],
       [4.47846097e+01, 4.00000000e+00, 0.00000000e+00, 1.20000000e+01,
        3.21539031e+00, 1.60000000e+01],
       [5.97128129e+01, 2.09541178e-31, 8.00000000e+00, 1.23259516e-32,
        4.28718708e+00, 0.00000000e+00]])

In [97]:
km = KMeans(n_clusters=2, random_state=5).fit(np.array(list(psds.values())))

In [98]:
np.array(list(psds.keys())).shape

(924,)

In [99]:
km.predict(np.array([psds[s] for s in states])).shape

(924,)

In [100]:
np.abs(km.predict(np.array([psds[s] for s in states])) - np.array([int(s in deviations) for s in states])).sum() / len(states)

np.float64(0.2878787878787879)

In [101]:
km.cluster_centers_

array([[13.55294118, 13.55294118, 13.55294118, 13.55294118, 13.55294118,
         8.47058824],
       [ 7.78378378,  7.78378378,  7.78378378,  7.78378378,  7.78378378,
        66.16216216]])

In [102]:
psd([1, 1, 1, 0, 0, 0])

array([1.60000000e+01, 6.16297582e-32, 4.00000000e+00])

In [107]:
psd([1, 1, 0, 1, 0, 0] * 4).round(2) / np.linalg.norm(psd([1, 1, 0, 1, 0, 0] * 4))

array([0.        , 0.        , 0.        , 0.30151134, 0.        ,
       0.        , 0.        , 0.90453403, 0.        , 0.        ,
       0.        , 0.30151134])

In [108]:
psd([1, 1, 0, 1, 0, 0]) / np.linalg.norm(psd([1, 1, 0, 1, 0, 0]))

array([0.30151134, 0.90453403, 0.30151134])

In [114]:
norm_psd([1, 1, 0, 1, 0, 0])

array([0.30151134, 0.90453403, 0.30151134])

In [123]:
print(norm_psd([1, 1, 0, 0]))
print(norm_psd([1, 1, 1, 0, 0, 0]))
print(norm_psd([1] * 8 + [0] * 8).round(3))

[1. 0.]
[9.70142500e-01 3.73685298e-33 2.42535625e-01]
[0.99  0.    0.122 0.    0.055 0.    0.039 0.   ]


In [169]:
# n is negative (no deviatoin), p is positive (deviation)
n_n = []
n_p = []
p_p = []
for s1, s2 in pairs:
    # dist = np.dot(norm_psds[s1], norm_psds[s2])
    dist = np.linalg.norm(norm_psds[s1] - norm_psds[s2])
    if s1 in deviations and s2 in deviations:
        p_p.append(dist)
    elif s1 not in deviations and s2 not in deviations:
        n_n.append(dist)
    else:
        n_p.append(dist)

In [170]:
print(f'avg nondev-nondev norm dist: {np.mean(n_n)}')
print(f'avg dev-dev norm dist: {np.mean(p_p)}')
print(f'avg nondev-dev norm dist {np.mean(n_p)}')
print(f'std nondev-nondev norm dist: {np.std(n_n)}')
print(f'std dev-dev norm dist: {np.std(p_p)}')
print(f'std nondev-dev norm dist {np.std(n_p)}')

avg nondev-nondev norm dist: 0.8962290881795624
avg dev-dev norm dist: 0.5595668149214151
avg nondev-dev norm dist 0.9478745279246524
std nondev-nondev norm dist: 0.3053703592837378
std dev-dev norm dist: 0.29628278567305927
std nondev-dev norm dist 0.2112607910229353


In [ ]:
km = KMeans(n_clusters=2, random_state=4).fit(np.array(list(norm_psds.values()))) # this doesn't really help here

In [136]:
np.abs(km.predict(np.array([norm_psds[s] for s in states])) - np.array([int(s in deviations) for s in states])).sum() / len(states)

np.float64(0.2532467532467532)

In [177]:
msr_signs =  {s: -1 + 2 * (mag.count_half_magnetization(s) % 2) for s in states}
o_o = []
e_e = []
o_e = []
for s1, s2 in pairs:
    # dist = np.dot(norm_psds[s1], norm_psds[s2])
    dist = np.linalg.norm(norm_psds[s1] - norm_psds[s2])
    if msr_signs[s1] == 1 and msr_signs[s2] == 1:
        o_o.append(dist)
    elif msr_signs[s1] == -1 and msr_signs[s2] == -1:
        e_e.append(dist)
    else:
        o_e.append(dist)

In [178]:
print('vector dist')
print(f'avg, std odd-odd dist: {np.mean(o_o), np.std(o_o)}')
print(f'avg, std even-even dist: {np.mean(e_e), np.std(e_e)}')
print(f'avg, std even-odd dist: {np.mean(o_e), np.std(o_e)}')

vector dist
avg, std odd-odd dist: (np.float64(0.9417189769554595), np.float64(0.3623249768426503))
avg, std even-even dist: (np.float64(0.7777204679074261), np.float64(0.2962530189649732))
avg, std even-odd dist: (np.float64(0.9342336854223101), np.float64(0.21406812066650646))


In [173]:
np.dot(norm_psds[2 ** 6 - 1], norm_psds[219])

np.float64(0.38235955645093617)

In [160]:
0b11011011

219

In [179]:
np.fft.rfft([1, 0, 1, 0])

array([2.+0.j, 0.+0.j, 2.+0.j])